# Chord Graph Data Processing

Starting with the raw sqlite database and the website classifications obtained:

In [42]:
import numpy as np
import pandas as pd
import sqlite3
from itertools import combinations

In [2]:
# Connect to SQLite database
con = sqlite3.connect("../data/top_100_shallow_1.sqlite")
cur = con.cursor()

In [5]:
# Read from websites_classification.csv
websites = pd.read_csv('../data/websites-classification.csv')

In [6]:
websites.head()

,url,class
0,http://google.com,Search Engines and Portals
1,http://youtube.com,Streaming Media
2,http://facebook.com,Social Networking
3,http://msn.com,Search Engines and Portals
4,http://yelp.com,Reference


In [7]:
_W = pd.read_sql_query('SELECT DISTINCT * FROM profile_cookies', con)

_W.rename(columns={'page_url': 'url'}, inplace=True)

In [8]:
_W.head()

,id,crawl_id,url,baseDomain,name,value,host,path,expiry,accessed,creationTime,isSecure,isHttpOnly
0,1,1,http://google.com,google.com,PREF,ID=1111111111111111:FF=0:TM=1436906302:LM=1436...,.google.com,/,1499978320,1436906320429022,1436906320429022,0,0
1,2,1,http://google.com,google.com,NID,69=QjonilIW3NrJTI3h12fdRQ3Gdy2g6DifOxZwlZS9X_v...,.google.com,/,1452717520,1436906320704467,1436906320429855,0,1
2,3,1,http://google.com,google.com,OGPC,5061451-1:,.google.com,/,1439498321,1436906321416369,1436906321416369,0,0
3,4,1,http://youtube.com,youtube.com,VISITOR_INFO1_LIVE,t4rCUKtDEzQ,.youtube.com,/,1457944310,1436906330561222,1436906330561222,0,1
4,5,1,http://youtube.com,youtube.com,PREF,f5=30,.youtube.com,/,1499978333,1436906333416922,1436906333416922,0,0


In [9]:
local_storage = pd.merge(_W, websites, on='url')
local_storage.head()

,id,crawl_id,url,baseDomain,name,value,host,path,expiry,accessed,creationTime,isSecure,isHttpOnly,class
0,1,1,http://google.com,google.com,PREF,ID=1111111111111111:FF=0:TM=1436906302:LM=1436...,.google.com,/,1499978320,1436906320429022,1436906320429022,0,0,Search Engines and Portals
1,2,1,http://google.com,google.com,NID,69=QjonilIW3NrJTI3h12fdRQ3Gdy2g6DifOxZwlZS9X_v...,.google.com,/,1452717520,1436906320704467,1436906320429855,0,1,Search Engines and Portals
2,3,1,http://google.com,google.com,OGPC,5061451-1:,.google.com,/,1439498321,1436906321416369,1436906321416369,0,0,Search Engines and Portals
3,4,1,http://youtube.com,youtube.com,VISITOR_INFO1_LIVE,t4rCUKtDEzQ,.youtube.com,/,1457944310,1436906330561222,1436906330561222,0,1,Streaming Media
4,5,1,http://youtube.com,youtube.com,PREF,f5=30,.youtube.com,/,1499978333,1436906333416922,1436906333416922,0,0,Streaming Media


In [22]:
pre_network_df = local_storage[['url', 'baseDomain', 'class']].drop_duplicates()
print(pre_network_df.shape)
pre_network_df.head(8)

(1419, 3)


,url,baseDomain,class
0,http://google.com,google.com,Search Engines and Portals
3,http://youtube.com,youtube.com,Streaming Media
5,http://youtube.com,doubleclick.net,Streaming Media
6,http://youtube.com,pointroll.com,Streaming Media
7,http://youtube.com,mookie1.com,Streaming Media
10,http://youtube.com,scorecardresearch.com,Streaming Media
12,http://youtube.com,voicefive.com,Streaming Media
16,http://facebook.com,facebook.com,Social Networking


In [23]:
# Group by baseDomain
by_domain = pre_network_df.groupby('baseDomain')
domain_urls = by_domain['url'].nunique()
domain_classes = by_domain['class'].nunique()
domain_classes.sort_values(ascending=False)

baseDomain
google.com               14
doubleclick.net          13
bluekai.com              12
advertising.com          11
quantserve.com           11
rubiconproject.com       11
rlcdn.com                11
openx.net                10
agkn.com                 10
scorecardresearch.com    10
adtechus.com              9
rfihub.com                9
twitter.com               9
casalemedia.com           9
demdex.net                9
facebook.com              9
turn.com                  9
yahoo.com                 9
pubmatic.com              9
nexac.com                 9
mathtag.com               9
bidswitch.net             8
revsci.net                8
amazon-adsystem.com       8
tapad.com                 8
adsrvr.org                8
adadvisor.net             8
imrworldwide.com          8
exelator.com              7
tubemogul.com             7
                         ..
p161.net                  1
dose.com                  1
sstatic.net               1
flashtalking.com          1
flickr.co

Let's create an Adjacency Matrix for the different classes, based on how many share a common cookie `baseDomain`

In [24]:
class_dict = { label: ind for ind, label in enumerate(websites['class'].unique()) }
class_dict

{'Adult Materials': 13,
 'Arts and Culture': 18,
 'Auction': 6,
 'Business': 11,
 'Education': 22,
 'Entertainment': 5,
 'File Sharing and Storage': 15,
 'Finance and Banking': 9,
 'Health and Wellness': 17,
 'Information Technology': 8,
 'Job Search': 19,
 'News and Media': 12,
 'Newsgroups and Message Boards': 16,
 'Personal Websites and Blogs': 7,
 'Political Organizations': 20,
 'Reference': 3,
 'Search Engines and Portals': 0,
 'Shopping': 4,
 'Social Networking': 2,
 'Sports': 14,
 'Streaming Media': 1,
 'Travel': 21,
 'Web Hosting': 10}

In [60]:
def add_json(c1, c2):
    ''' Add an edge, accounting for possible inexistences of keys '''
    if c1 in pre_json:
        if c2 in pre_json[c1]:
            pre_json[c1][c2] += 1
        else:
            pre_json[c1][c2] = 1
    else:
        pre_json[c1] = {c2: 1}
            
# Initialize Matrix and nested dictionary (precursor for final JSON)
adjacency_matrix = np.zeros( (len(class_dict),len(class_dict)) ) # list of edge - tuples
pre_json = {}

# For each baseDomain
for bd, df in by_domain['class']:
    cluster = df.values  # cluster, all connected by the same baseDomain `bd`
    edges = combinations(df.values, 2) # iterable of all possible 2 element combinations
    for c1, c2 in edges:
        add_json(c1, c2)
        adjacency_matrix[class_dict[c1], class_dict[c2]] += 1
        if c1 != c2:  # if not diagonal add the reverse combination
            adjacency_matrix[class_dict[c2], class_dict[c1]] += 1
            add_json(c2, c1)


# Create final JSON object with desired format         
adjacency_json = {}
for c1, item in pre_json.items():
    adjacency_json[c1] = [ [c2,count] for c2, count in item.items() ]

In [54]:
adjacency_matrix

array([[  16.,    4.,    2.,   65.,   47.,  122.,    1.,    6.,   29.,
          10.,    7.,    1.,  310.,   38.,   44.,   38.,   12.,   16.,
           9.,    4.,    0.,    5.,    0.],
       [   4.,    0.,    1.,    8.,    6.,   34.,    1.,    5.,    5.,
           2.,    2.,    1.,   45.,    5.,    8.,    6.,    3.,    4.,
           4.,    2.,    0.,    0.,    0.],
       [   2.,    1.,    0.,    5.,    4.,   15.,    0.,    3.,    1.,
           0.,    2.,    0.,   14.,    3.,    1.,    4.,    0.,    0.,
           0.,    2.,    0.,    0.,    0.],
       [  65.,    8.,    5.,   48.,   80.,  176.,    2.,    9.,   39.,
          12.,   13.,    8.,  499.,   84.,   54.,   62.,   29.,   32.,
          17.,   17.,    0.,    9.,    0.],
       [  47.,    6.,    4.,   80.,   14.,  102.,    0.,    8.,   19.,
           9.,    6.,    3.,  267.,   45.,   31.,   32.,   12.,   16.,
          14.,   13.,    0.,    2.,    0.],
       [ 122.,   34.,   15.,  176.,  102.,  206.,    3.,   37.,   81.,

**This dataset has the necessary format for the CHORD DIAGRAM**  
  
*NOTE - The adjacency matrix has big numbers due to the fact that cookies are not unique, and some sites contain numerous cookies which they may share with other url's (within or in other) url classes.*

In [61]:
adjacency_json

{'Adult Materials': [['Finance and Banking', 3],
  ['Health and Wellness', 18],
  ['File Sharing and Storage', 42],
  ['Arts and Culture', 15],
  ['News and Media', 315],
  ['Reference', 84],
  ['Sports', 33],
  ['Travel', 2],
  ['Streaming Media', 5],
  ['Entertainment', 103],
  ['Shopping', 45],
  ['Information Technology', 25],
  ['Personal Websites and Blogs', 6],
  ['Social Networking', 3],
  ['Business', 3],
  ['Adult Materials', 11],
  ['Web Hosting', 5],
  ['Newsgroups and Message Boards', 17],
  ['Job Search', 10],
  ['Search Engines and Portals', 38]],
 'Arts and Culture': [['Finance and Banking', 1],
  ['Health and Wellness', 9],
  ['File Sharing and Storage', 10],
  ['Reference', 17],
  ['News and Media', 83],
  ['Arts and Culture', 1],
  ['Sports', 10],
  ['Streaming Media', 4],
  ['Entertainment', 34],
  ['Shopping', 14],
  ['Information Technology', 6],
  ['Personal Websites and Blogs', 1],
  ['Business', 1],
  ['Adult Materials', 15],
  ['Newsgroups and Message Boards',

**This dataset has the intended structure for the ADJACENCY_MATRIX**

### Save files to `data/processed`

In [65]:
with open('../data/processed/chord_diagram_data.txt', 'w') as f:
    f.write(str(adjacency_matrix))

with open('../data/processed/adjacency_viz_matrix_data.txt', 'w') as f:
    f.write(str(adjacency_json))